In [ ]:
import os
import sys
import f90nml
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd()
sys.path.append(os.path.expandvars("$CODE/NEO-RT/python"))

runs = ['0p10', '0p12', '0p14', '0p16', '0p18',
        '0p20', '0p22', '0p24', '0p26', '0p28',
        '0p30', '0p32', '0p34', '0p36', '0p38',
        '0p40', '0p42', '0p44', '0p46', '0p48',
        '0p50', '0p52', '0p54', '0p56', '0p58',
        '0p60', '0p62', '0p64', '0p66', '0p68',
        '0p70', '0p72', '0p74', '0p76', '0p78',
        '0p80', '0p82', '0p83', '0p85', '0p87',
        '0p90', '0p95', '0p97']

run_dir = os.path.join(os.path.expandvars("$CODE/tests/NEO-RT/benchmark_with_NEO_2/RUN"))
if os.path.isdir(run_dir):
    os.system(f'rm -r {run_dir}')
os.makedirs(run_dir)

In [ ]:
## Generate torque runs

template = f90nml.read('neort_torque.in.template')
for run in runs:
    nml = template
    s = float(run.replace('p', '.'))
    nml['params']['s'] = s
    nml.write(os.path.join(run_dir,f'torque{run}.in'), force=True)

In [ ]:
## Load NEO-2 output

neo2_dir = '/proj/plasma/DATA/DEMO/NEO-2/comparison_vary_coilwidth_MARS_NEO2/lagrange_run_001'
neo_input = f90nml.read(os.path.join(neo2_dir, 'neo.in'))
neo2_input = f90nml.read(os.path.join(neo2_dir, 'neo2.in'))
neo2_output_file = os.path.join(neo2_dir, 'neo2_multispecies_out.h5')
neo2_output = nc.Dataset(neo2_output_file)


In [ ]:
# Convert NEO-2 profiles and link to magnetic fields
from neo_2_profile_to_neo_rt_profile import neo_2_profile_to_neo_rt_profile
neo2_profiles_file = os.path.join(neo2_dir, 'multi_spec_demo.in')
os.chdir(run_dir)
neo_2_profile_to_neo_rt_profile(
        neo2_profiles_file, neo2_output_file, s_min=0.01, s_max=0.972, number_s_points=32)
axisymmetric = os.path.join(neo2_dir, 'axi.bc')
os.system(f'ln -fs {axisymmetric} in_file')
pert = os.path.join(neo2_dir, 'pert.bc')
os.system(f'ln -fs {pert} in_file_pert')
os.chdir(cwd)

In [ ]:
## Start torque runs

neort_exe = os.path.expandvars("$CODE/NEO-RT/build/neo_rt.x")
os.chdir(run_dir)
for run in runs:
    os.system(f'{neort_exe} torque{run}')
os.chdir(cwd)

In [ ]:
## Load and plot output for ion NTV torque density for NEO-RT and NEO-2

from util import load_torque

NM_IN_DYNCM = 1e-7
CM3_IN_M3 = 1e-6
TORQUE_DENSITY_CGS_TO_SI = NM_IN_DYNCM/CM3_IN_M3  # dyn*cm/cm^3 to Nm/m^3

neort = []
for run in runs:
    neort.append(load_torque(os.path.join(run_dir,f'torque{run}'), neo2_output))

Tphi_neort = np.array([v['Tphi'] for v in neort])
dVds = np.array([v['dVds'] for v in neort])


Tphi_neo2 = neo2_output['TphiNA_spec'][:,1]*TORQUE_DENSITY_CGS_TO_SI
s_neort = np.array([v['s'] for v in neort])
s_neo2 = neo2_output['boozer_s'][:]

plt.figure(figsize=(12,6))
plt.plot(s_neo2, Tphi_neo2, label='NEO-2')
plt.plot(s_neort, Tphi_neort, 'x', label='NEO-RT')
plt.yscale('symlog', linthresh=1e-10)
plt.xlabel('Normalized toroidal flux s_tor')
plt.ylabel('Tphi [Nm/m^3]')
plt.legend()
plt.title('Ion NTV torque density Tphi')
plt.savefig('Tphi_comparison.png')

In [ ]:
## Generate transport runs

template = f90nml.read('neort_transport.in.template')
for run in runs:
    nml = template
    s = float(run.replace('p', '.'))
    nml['params']['s'] = s
    nml.write(os.path.join(run_dir,f'transport{run}.in'), force=True)

In [ ]:
## Start transport runs

neort_exe = os.path.expandvars("$CODE/NEO-RT/build/neo_rt.x")
os.chdir(run_dir)
for run in runs:
    os.system(f'{neort_exe} transport{run}')
os.chdir(cwd)

In [ ]:
## Load NEO-RT output for non-axisymmetric ion transport coefficient D11_NA

from util import load_transport

neort = []
for run in runs:
    neort.append(load_transport(os.path.join(run_dir,f'transport{run}'), neo2_output))

s_neort = np.array([v['s'] for v in neort])
D11_neort = np.array([v['D11'] for v in neort])
s_neo2 = neo2_output['boozer_s'][:]
D11_neo2_ion = neo2_output['D11_NA'][:][:,3]

plt.figure(figsize=(12,6))
plt.plot(s_neo2, D11_neo2_ion, label='NEO-2')
plt.plot(s_neort, D11_neort, 'x', label='NEO-RT')
plt.yscale('symlog', linthresh=1e-10)
plt.xlabel('Normalized toroidal flux s_tor')
plt.ylabel('D11_NA')
plt.legend()
plt.title('Non-axisymmetric ion transport coefficient D11_NA')
plt.savefig('D11_NA_comparison.png')

In [ ]:
# Prepare resline test
template = f90nml.read('neort_resline.in.template')
template.write(os.path.join(run_dir,f'resline.in'), force=True)

In [ ]:
# Run resline test
neort_exe = os.path.expandvars("$CODE/NEO-RT/build/neo_rt.x")
os.chdir(run_dir)
os.system(f'{neort_exe} resline')
os.chdir(cwd)

In [ ]:
data = np.loadtxt(os.path.join(run_dir,'resline_resline_pct.out'))

v_ov_vth = data[:,0]
branch_index = data[:,1].astype(int)
eta_res = data[:,2]
eta_0 = data[:,3][0]
eta_1 = data[:,4][0]

plt.figure()
for i in range(1, max(branch_index)+1):
    plt.plot(v_ov_vth[branch_index==i],
             eta_res[branch_index==i], label='branch %d' % i)
plt.plot([0.0, max(data[:,0])], [eta_0, eta_0], '--',
    label='lower eta boundary')
plt.plot([0.0, max(data[:,0])], [eta_1, eta_1], '--',
    label='upper eta boundary')
plt.xlabel('Normalized velocity v/v_th')
plt.ylabel('Normalized magnetic moment eta')
plt.legend()
plt.title('Phase-space resonance (example)')